In [22]:
import gradio as gr
import ast
from pynput import keyboard
import json
import os

In [23]:
# File path for the macros
macros_file_path = 'macros.json'

In [24]:
def update_macros(action, keys, macro_to_delete, timing, current_macros):
    # Check if current_macros is a string representation of a dictionary, and convert it if so
    if isinstance(current_macros, str) and current_macros.strip():
        # Safely convert the string representation back to a dictionary
        current_macros = ast.literal_eval(current_macros)
    else:
        # Initialize as an empty dictionary if not a valid string representation
        current_macros = {}
    
    # Check if a user entered a macro to delete
    if macro_to_delete and macro_to_delete in current_macros:
        del current_macros[macro_to_delete]  
    elif action and keys:  # Ensure non-empty input
        # Update or add the macro definition
        newlist = key_name.copy()
        key_codes = [str(i) for i in key_list.copy()]
        current_macros[action] = {
            'name': action,
            'char_keys': newlist,
            'keycodes': [key_codes],  # Store virtual key codes
            'delay': timing
        }
        # current_macros[action] = {'keys': newkey, 'timing': timing}
    # Format the macros for display in a readable format
    macros_text = ""
    for act, vals in current_macros.items():
        macros_text += f"{act}\t\t=>\t\t{vals['char_keys']}\t\t({vals['delay']} second delay)\n"

    clear_list()
    
    # Check if the file exists
    if not os.path.exists(macros_file_path):
        # If not, create an empty macros file with the basic structure
        with open(macros_file_path, 'w') as file:
            json.dump({"macros": []}, file, indent=4)
        macros = {}
    else:
        # If it exists, load the existing macros
        with open(macros_file_path, 'r') as file:
            data = json.load(file)
            macros = {macro['name']: macro for macro in data.get("macros", [])}
    # Save the updated macros to a JSON file
    with open(macros_file_path, 'w') as file:
        json.dump({"macros": list(current_macros.values())}, file, indent=4)

    # Return both the formatted string for display and the updated macros dictionary, Clear the display for action_input, keys_input, and macro_to_delte
    return macros_text, str(current_macros), "", "", 0, "Keys to Simulate"

In [25]:
key_list = []
key_name = []


def read_keys():
    def on_press(key):
        
        try:
            print('alphanumeric key {0} pressed'.format(key.char))
            if key == keyboard.Key.esc:
                return False
        except AttributeError:
            print('special key {0} pressed'.format(key))
        
    def on_release(key):
        try:
            key_list.append(key.char)
            key_name.append(key.char)
        except AttributeError:
            if key == keyboard.Key.esc:
                return False
            key_str = getattr(key, 'name', str(key))
            key_list.append(key.value)
            key_name.append(key_str)
        print(key_list)
    with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
        listener.join()

    return "Ended Recording"

def clear_list():
    key_list.clear()
    key_name.clear()


In [26]:
def update_key_btn():
    return "Now Recording Press The 'esc' Key To Stop"

In [27]:
with gr.Blocks() as settings_page:
    gr.Markdown("# Macro Setting")
    gr.Markdown("""
    Configure your audio command macros here. Enter a speech command, enter your key sequence, and set the timing.
    """)

    with gr.Row():
        # Create a textbox for the speech command
        action_input = gr.Textbox(label="Speech Command") 
        delays_input = gr.Slider(minimum=0, maximum=5, step= 0.1, label="Delays Between Each KeyPress")
        # Create a textbox for the key
        keys_input = gr.Button("Keys to Simulate")

    # Gets the macro the user wants to delete
    macro_to_delete = gr.Textbox(label="Type the Speech Command to Delete")
    # Creates a button to update or add the new settings
    update_btn = gr.Button("Update/Add Macro") 
    
    # Display for the current macros
    current_macros_display = gr.Textbox(label="Current Macros", value="", lines=10, interactive=False) 
    macros_state = gr.State('{}')
    # Runs when the button is clicked updating the current macros display
    keys_input.click(update_key_btn, outputs=keys_input)
    keys_input.click(read_keys, outputs=keys_input)
    update_btn.click(update_macros, 
                     inputs=[action_input, keys_input,macro_to_delete, delays_input, macros_state], 
                     outputs=[current_macros_display, macros_state, action_input, macro_to_delete, delays_input, keys_input])


In [28]:
settings_page.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


special key Key.ctrl_l pressed
alphanumeric key  pressed
['\x03']
['\x03', <162>]
special key Key.esc pressed
special key Key.ctrl_l pressed
[<162>]
alphanumeric key c pressed
[<162>, 'c']
special key Key.esc pressed
special key Key.ctrl_l pressed
alphanumeric key  pressed
[<162>]
[<162>, 'v']
special key Key.esc pressed
